In [ ]:
import numpy as np
import cv2
from skimage.registration import optical_flow_tvl1
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter
from scipy.ndimage import gaussian_filter1d
import os


In [ ]:


video_folder = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good'

# List all files in the directory
files = os.listdir(video_folder)

# Filter to keep only .avi files
videos = [f for f in files if f.endswith('.avi')]

print(videos)


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.registration import optical_flow_tvl1
from skimage import filters
from skimage.transform import resize

In [ ]:
import numpy as np
import cv2

# Read video
cap = cv2.VideoCapture('/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/0_MT_1uL_correlation.avi')

# Predefine the list of numbers to check
check_numbers = np.arange(1, 101)  # Check numbers from 1 to 100

# Prepare an empty list to store the correlation lengths
correlation_lengths = []

for num in check_numbers:
    try:
        # Insert the calculation logic here

        # Assume 'correlation_length' is the result of the calculation
        correlation_lengths.append(correlation_lengths)
    except Exception as e:
        print(f"An error occurred for number {num}: {str(e)}")

# Print out the results
for num, length in zip(check_numbers, correlation_lengths):
    print(f"Correlation length for number {num}: {length}")


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.registration import optical_flow_tvl1
from skimage import filters
from skimage.transform import resize

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.registration import optical_flow_tvl1
from skimage import filters
from skimage.transform import resize

# Initial setup
saveimg = False
sample = '235_MT_correlation_15ul'
vidpath = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/'
vidname = vidpath + sample + '.avi'
EdgeLength = 40
smooth = True
smooth_len = 10
pxlen = .43
vscale = 1
tmin = 10
tmax = 100
delta_T = 1
step = 5
tseries = range(tmin, tmax+1, step)

# Open the video
cap = cv2.VideoCapture(vidname)

if not cap.isOpened():
    raise Exception(f"Could not open video file {vidname}")

# Define the range of the video we are interested in
ret, frame1 = cap.read()
yrange = range(851,1051)
xrange = range(frame1.shape[1])

# Get the total number of frames
Ttot = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Read the initial frame
cap.set(cv2.CAP_PROP_POS_FRAMES, 100)  # frame numbering in OpenCV starts from 0
ret, frame1 = cap.read()

# Convert the color image to grayscale
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Crop and normalize the image
im1 = cv2.normalize(gray1[yrange, :][:, xrange], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

print("Video has been successfully read and initial frame has been processed.")


In [ ]:
# Define X1 and Y1 similar to your MATLAB code
X1, Y1 = np.meshgrid(np.arange(EdgeLength / 2, len(xrange) - EdgeLength / 2, EdgeLength),
                     np.arange(EdgeLength / 2, len(yrange) - EdgeLength / 2, EdgeLength))

# Initialize correlation length array
corr_len = np.zeros(len(tseries))

# Loop over the time series
for i, tt in enumerate(tseries):
    # Read frames tt and tt+delta_T
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt)
    _, frame1 = cap.read()
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt+delta_T)
    _, frame2 = cap.read()

    # Convert to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]

    # Normalize
    im1 = cv2.normalize(gray1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    im2 = cv2.normalize(gray2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Compute the optical flow
    flow = optical_flow_tvl1(im1, im2)
    VX = -flow[..., 0] / delta_T
    VY = -flow[..., 1] / delta_T

    # Apply Gaussian smoothing if desired
    if smooth:
        VX = filters.gaussian(VX, sigma=smooth_len)
        VY = filters.gaussian(VY, sigma=smooth_len)

    # Subtract the mean
    VX -= VX.mean()
    VY -= VY.mean()

    print(f"Processed time step {tt}.")


In [ ]:
# Define the criteria for the ECC algorithm
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100, 0.001)

# Initialize the warp matrix with identity transformation
warp_matrix = np.eye(2, 3, dtype=np.float32)

# Apply the ECC algorithm
(cc, warp_matrix) = cv2.findTransformECC(im1, im2, warp_matrix, cv2.MOTION_EUCLIDEAN, criteria)

# Use warpAffine to warp the second image
im2_aligned = cv2.warpAffine(im2, warp_matrix, (im2.shape[1], im2.shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

# Now let's calculate the optical flow between the original and warped images
flow = cv2.calcOpticalFlowFarneback(im1, im2_aligned, None, pyr_scale=0.5, levels=5, winsize=11, iterations=5, poly_n=5, poly_sigma=1.2, flags=0)

VX = flow[..., 0]
VY = flow[..., 1]

# Create PX and PY by element-wise multiplication of im1 and VX, VY respectively
PX = im1 * VX
PY = im1 * VY

print(f"Shape of VX: {VX.shape}, Shape of VY: {VY.shape}")
print(f"Shape of PX: {PX.shape}, Shape of PY: {PY.shape}")


In [ ]:
from scipy.ndimage import gaussian_filter

if smooth:
    VX_smooth = gaussian_filter(VX, smooth_len)
    VY_smooth = gaussian_filter(VY, smooth_len)
    print("VX and VY have been smoothed successfully.")
else:
    VX_smooth = VX
    VY_smooth = VY
    print("No smoothing applied.")

VX_mean_subtracted = VX_smooth - np.mean(VX_smooth)
VY_mean_subtracted = VY_smooth - np.mean(VY_smooth)

print("Mean subtracted from VX and VY successfully.")

PX = im1 * VX_mean_subtracted
PY = im1 * VY_mean_subtracted

print("PX and PY have been calculated.")

In [ ]:
# Step 7: Resize and rescale UX and UY
UX = cv2.resize(PX, (Y1.shape[1], X1.shape[0])) * vscale
UY = cv2.resize(PY, (Y1.shape[1], X1.shape[0])) * vscale

print("UX and UY have been resized and rescaled.")

# Step 8: Calculate the mean, smoothed data, and product of smoothed data
PXmean = np.mean(PX, axis=0)
PXsmth = gaussian_filter1d(PXmean, smooth_len)
PXsmth = PXsmth / np.max(np.abs(PXsmth))

PXprod = PXsmth[:-1] * PXsmth[1:]

print("Calculated the mean, smoothed data, and product of smoothed data.")



In [ ]:
# Step 9: Find zero-crossings in PXprod
PXzc = np.where(np.diff(np.sign(PXprod)))[0]

print("Found zero-crossings in PXprod.")

# Step 10: Calculate the first difference of PXsmth
PXd1 = np.diff(PXsmth, n=1)

print("Calculated the first difference of PXsmth.")



In [ ]:
PXzc

In [ ]:
# Step 11: Identify sinks
PXsink = [ii for ii in PXzc if PXd1[ii]<0]

print(f"Identified {len(PXsink)} sinks.")


In [ ]:
PXsink

In [ ]:
print("Peaks:", peaks)
print("Wells:", wells)
# Step 13: Calculate peaks and wells
peak_indices = []
well_indices = []
for ss in range(len(PXsrc)-1):
    range_min, range_max = PXsrc[ss] - 1, PXsrc[ss+1] - 1
    if range_min < range_max:
        peak_indices.append(np.argmax(PXsmth[range_min:range_max+1]) + range_min)
        well_indices.append(np.argmin(PXsmth[range_min:range_max+1]) + range_min)

print(f"Identified {len(peak_indices)} peak indices and {len(well_indices)} well indices.")
# Step 14: Calculate gaps
gaps = [well_indices[ii] - peak_indices[ii] for ii in range(len(peak_indices)) if well_indices[ii] - peak_indices[ii] > 0]

print(f"Identified {len(gaps)} gaps.")


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.registration import optical_flow_tvl1
from skimage import filters
from scipy.ndimage import gaussian_filter1d

def open_video(vidname):
    cap = cv2.VideoCapture(vidname)
    if not cap.isOpened():
        raise Exception(f"Could not open video file {vidname}")
    return cap

def process_initial_frame(cap, xrange, yrange):
    ret, frame1 = cap.read()
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    im1 = cv2.normalize(gray1[yrange, :][:, xrange], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    return im1

def compute_optical_flow(cap, tt, delta_T, yrange, xrange, smooth, smooth_len):
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt)
    _, frame1 = cap.read()
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt+delta_T)
    _, frame2 = cap.read()
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]
    im1 = cv2.normalize(gray1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    im2 = cv2.normalize(gray2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    flow = optical_flow_tvl1(im1, im2)
    VX = -flow[..., 0] / delta_T
    VY = -flow[..., 1] / delta_T
    if smooth:
        VX = filters.gaussian(VX, sigma=smooth_len)
        VY = filters.gaussian(VY, sigma=smooth_len)
    VX -= VX.mean()
    VY -= VY.mean()
    return VX, VY

def compute_PX_and_PY(im1, VX, VY):
    PX = im1 * VX
    PY = im1 * VY
    return PX, PY

def compute_correlation_lengths(PX, PXsrc, PXsmth, pxlen):
    peaks = []
    wells = []
    for ss in range(len(PXsrc)-1):
        range_min, range_max = PXsrc[ss], PXsrc[ss+1]
        if range_min >= range_max:
            print(f"Empty range at index {ss} with range_min: {range_min}, range_max: {range_max}")
            continue
        peaks.append(np.argmax(PXsmth[range_min:range_max+1]))
        wells.append(np.argmin(PXsmth[range_min:range_max+1]))
    gaps = [wells[ii] - peaks[ii] for ii in range(len(peaks)) if wells[ii] - peaks[ii] > 0]
    corr_len = np.mean(gaps) * pxlen
    return corr_len


In [ ]:
sample = '235_MT_correlation_15ul'
vidpath = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/'
vidname = vidpath + sample + '.avi'

cap = open_video(vidname)

im1 = process_initial_frame(cap, xrange, yrange)

X1, Y1 = np.meshgrid(np.arange(EdgeLength / 2, len(xrange) - EdgeLength / 2, EdgeLength),
                     np.arange(EdgeLength / 2, len(yrange) - EdgeLength / 2, EdgeLength))

corr_len = np.zeros(len(tseries))

for i, tt in enumerate(tseries):
    VX, VY = compute_optical_flow(cap, tt, delta_T, yrange, xrange, smooth, smooth_len)
    PX, PY = compute_PX_and_PY(im1, VX, VY)
    corr_len[i] = compute_correlation_lengths(PX, PXsrc, PXsmth, pxlen)
    print(f"Correlation length at time step {tt}: {corr_len[i]}")


In [ ]:
im1 = read_and_preprocess_frame(cap, 100, yrange, xrange)

# Loop over the time series
for tt in tseries:
    # Read frames tt and tt+delta_T
    im1 = read_and_preprocess_frame(cap, tt, yrange, xrange)
    im2 = read_and_preprocess_frame(cap, tt+delta_T, yrange, xrange)

    # Compute the optical flow
    VX, VY = compute_optical_flow(im1, im2)

    # Smooth and zero mean
    VX, VY = smooth_and_zero_mean(VX, VY, smooth_len)

    # Compute PX and PY
    PX, PY = compute_PX_and_PY(im1, VX, VY)

    # Compute correlation lengths
    corr_len = compute_correlation_lengths(PX, PXsrc, PXsmth, pxlen)
    print(f"Correlation length at time step {tt}: {corr_len}")


In [ ]:
# Step 15: Plot the correlation length

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(tseries, corr_len, linewidth=5)
plt.xlabel('Time')
plt.ylabel('Correlation Length / um')
plt.grid(True)
plt.title('Correlation Length vs Time')
plt.savefig('CorrLength.png')
plt.show()


In [ ]:
# Import necessary libraries
import cv2
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks

# Set the parameters
filename = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/235_MT_correlation_15ul.avi'  # your video file
yrange = slice(851, 1050)
xrange = slice(1, 2048)  # change to match your video's width
tmin = 10
tmax = 100
delta_T = 1
step = 5
tseries = range(tmin, tmax+1, step)
pxlen = .43
smooth = True
smooth_len = 10

# Open the video file
cap = cv2.VideoCapture(filename)

# Initialize an empty list to store correlation lengths
corr_len = []

# Process each pair of frames
for tt in tseries:
    # Read and preprocess the frames
    im1 = read_and_preprocess_frame(cap, tt, yrange, xrange)
    im2 = read_and_preprocess_frame(cap, tt + delta_T, yrange, xrange)
    
    # Compute the optical flow
    VX, VY = compute_optical_flow(im1, im2)
    
    # Smooth and zero-mean the vectors
    VX, VY = smooth_and_zero_mean(VX, VY, smooth_len)
    
    # Compute PX and PY
    PX, PY = compute_PX_and_PY(im1, VX, VY)

    # Smooth the PX array for peak detection
    PXsmth = gaussian_filter1d(PX, 10, mode='constant', cval=0.0)
    
    # Find sources, sinks, and gaps
    PXsrc, PXsink, gaps = find_sources_sinks_and_gaps(PX, xrange, PXsmth)
    
    # Compute the correlation lengths
    Lx = compute_correlation_lengths(PX, PXsrc, PXsmth, pxlen)
    
    # Append the correlation length to the list
    corr_len.append(Lx)

    # Print the correlation lengths
    print(f"Correlation lengths for frame {tt} to {tt+delta_T}: Lx = {Lx}")
    
# Close the video file
cap.release()

# At this point, corr_len contains the correlation lengths for all frame pairs
# You can save it to a file or plot it as needed


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.registration import optical_flow_tvl1
from skimage import filters
from skimage.transform import resize

# Initial setup
saveimg = False
sample = '235_MT_correlation_15ul'
vidpath = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/'
vidname = vidpath + sample + '.avi'
EdgeLength = 40
smooth = True
smooth_len = 10
pxlen = .43
vscale = 1
tmin = 10
tmax = 100
delta_T = 1
step = 5
tseries = range(tmin, tmax+1, step)

# Open the video
cap = cv2.VideoCapture(vidname)

if not cap.isOpened():
    raise Exception(f"Could not open video file {vidname}")

# Define the range of the video we are interested in
ret, frame1 = cap.read()
yrange = range(851,1051)
xrange = range(frame1.shape[1])

# Get the total number of frames
Ttot = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Read the initial frame
cap.set(cv2.CAP_PROP_POS_FRAMES, 100)  # frame numbering in OpenCV starts from 0
ret, frame1 = cap.read()

# Convert the color image to grayscale
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Crop and normalize the image
im1 = cv2.normalize(gray1[yrange, :][:, xrange], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

print("Video has been successfully read and initial frame has been processed.")


# Define X1 and Y1 similar to your MATLAB code
X1, Y1 = np.meshgrid(np.arange(EdgeLength / 2, len(xrange) - EdgeLength / 2, EdgeLength),
                     np.arange(EdgeLength / 2, len(yrange) - EdgeLength / 2, EdgeLength))

# Initialize correlation length array
corr_len = np.zeros(len(tseries))

# Loop over the time series
for i, tt in enumerate(tseries):
    # Read frames tt and tt+delta_T
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt)
    _, frame1 = cap.read()
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt+delta_T)
    _, frame2 = cap.read()

    # Convert to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]

    # Normalize
    im1 = cv2.normalize(gray1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    im2 = cv2.normalize(gray2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Compute the optical flow
    flow = optical_flow_tvl1(im1, im2)
    VX = -flow[..., 0] / delta_T
    VY = -flow[..., 1] / delta_T

    # Apply Gaussian smoothing if desired
    if smooth:
        VX = filters.gaussian(VX, sigma=smooth_len)
        VY = filters.gaussian(VY, sigma=smooth_len)

    # Subtract the mean
    VX -= VX.mean()
    VY -= VY.mean()

    print(f"Processed time step {tt}.")


# Define the criteria for the ECC algorithm
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100, 0.001)

# Initialize the warp matrix with identity transformation
warp_matrix = np.eye(2, 3, dtype=np.float32)

# Apply the ECC algorithm
(cc, warp_matrix) = cv2.findTransformECC(im1, im2, warp_matrix, cv2.MOTION_EUCLIDEAN, criteria)

# Use warpAffine to warp the second image
im2_aligned = cv2.warpAffine(im2, warp_matrix, (im2.shape[1], im2.shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

# Now let's calculate the optical flow between the original and warped images
flow = cv2.calcOpticalFlowFarneback(im1, im2_aligned, None, pyr_scale=0.5, levels=5, winsize=11, iterations=5, poly_n=5, poly_sigma=1.2, flags=0)

VX = flow[..., 0]
VY = flow[..., 1]

# Create PX and PY by element-wise multiplication of im1 and VX, VY respectively
PX = im1 * VX
PY = im1 * VY

print(f"Shape of VX: {VX.shape}, Shape of VY: {VY.shape}")
print(f"Shape of PX: {PX.shape}, Shape of PY: {PY.shape}")


from scipy.ndimage import gaussian_filter

if smooth:
    VX_smooth = gaussian_filter(VX, smooth_len)
    VY_smooth = gaussian_filter(VY, smooth_len)
    print("VX and VY have been smoothed successfully.")
else:
    VX_smooth = VX
    VY_smooth = VY
    print("No smoothing applied.")

VX_mean_subtracted = VX_smooth - np.mean(VX_smooth)
VY_mean_subtracted = VY_smooth - np.mean(VY_smooth)

print("Mean subtracted from VX and VY successfully.")

PX = im1 * VX_mean_subtracted
PY = im1 * VY_mean_subtracted

print("PX and PY have been calculated.")

# Step 7: Resize and rescale UX and UY
UX = cv2.resize(PX, (Y1.shape[1], X1.shape[0])) * vscale
UY = cv2.resize(PY, (Y1.shape[1], X1.shape[0])) * vscale

print("UX and UY have been resized and rescaled.")

# Step 8: Calculate the mean, smoothed data, and product of smoothed data
PXmean = np.mean(PX, axis=0)
PXsmth = gaussian_filter1d(PXmean, smooth_len)
PXsmth = PXsmth / np.max(np.abs(PXsmth))

PXprod = PXsmth[:-1] * PXsmth[1:]

print("Calculated the mean, smoothed data, and product of smoothed data.")


# Step 9: Find zero-crossings in PXprod
PXzc = np.where(np.diff(np.sign(PXprod)))[0]

print("Found zero-crossings in PXprod.")

# Step 10: Calculate the first difference of PXsmth
PXd1 = np.diff(PXsmth, n=1)

print("Calculated the first difference of PXsmth.")


# Step 11: Identify sinks
PXsink = [ii for ii in PXzc if PXd1[ii]<0]

# Calculate the correlation distance between pixels
if len(PXsink) > 1:
    corr_distance = np.mean(np.diff(PXsink))
else:
    # If there's only one sink, correlation length is the width of the rectangle
    corr_distance = EdgeLength * pxlen

corr_len[tp] = corr_distance

print(f"Identified {len(PXsink)} sinks.")


# Step 13: Calculate peaks and wells
peak_indices = []
well_indices = []
for ss in range(len(PXsrc)-1):
    range_min, range_max = PXsrc[ss] - 1, PXsrc[ss+1] - 1
    if range_min < range_max:
        peak_indices.append(np.argmax(PXsmth[range_min:range_max+1]) + range_min)
        well_indices.append(np.argmin(PXsmth[range_min:range_max+1]) + range_min)

print(f"Identified {len(peak_indices)} peak indices and {len(well_indices)} well indices.")
# Step 14: Calculate gaps
gaps = [well_indices[ii] - peak_indices[ii] for ii in range(len(peak_indices)) if well_indices[ii] - peak_indices[ii] > 0]

print(f"Identified {len(gaps)} gaps.")


In [ ]:
%load_ext blackcellmagic


In [ ]:
import cv2
import numpy as np
from skimage.registration import optical_flow_tvl1
from skimage import filters
from scipy.ndimage import gaussian_filter, gaussian_filter1d

# Initialize an array to store correlation lengths for all pairs of frames
all_corr_len = np.zeros((len(tseries), len(tseries)))

# Initial setup
saveimg = False
sample = '235_MT_correlation_15ul'
vidpath = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/'
vidname = vidpath + sample + '.avi'
EdgeLength = 40
smooth = True
smooth_len = 10
pxlen = .43
vscale = 1
tmin = 10
tmax = 100
delta_T = 1
step = 5
tseries = range(tmin, tmax+1, step)

# Open the video
cap = cv2.VideoCapture(vidname)

if not cap.isOpened():
    raise Exception(f"Could not open video file {vidname}")

# Define the range of the video we are interested in
ret, frame1 = cap.read()
yrange = range(851, 1051)
xrange = range(frame1.shape[1])

# Nested loop to calculate correlation lengths for all pairs of frames
for i, tt1 in enumerate(tseries):
    for j, tt2 in enumerate(tseries):
        # Skip if the two frames are the same
        if i == j:
            continue

        # Read frames tt1 and tt2
        cap.set(cv2.CAP_PROP_POS_FRAMES, tt1)
        _, frame1 = cap.read()
        cap.set(cv2.CAP_PROP_POS_FRAMES, tt2)
        _, frame2 = cap.read()

        # Convert frames to grayscale and normalize
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]
        im1 = cv2.normalize(gray1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        im2 = cv2.normalize(gray2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        # Define X1 and Y1 similar to your MATLAB code
        X1, Y1 = np.meshgrid(np.arange(EdgeLength / 2, len(xrange) - EdgeLength / 2, EdgeLength),
                             np.arange(EdgeLength / 2, len(yrange) - EdgeLength / 2, EdgeLength))

        # Initialize correlation length array
        corr_len = np.zeros(len(tseries))

        # Loop over the time series
        for i, tt in enumerate(tseries):
            # Read frames tt and tt+delta_T
            cap.set(cv2.CAP_PROP_POS_FRAMES, tt)
            _, frame1 = cap.read()
            cap.set(cv2.CAP_PROP_POS_FRAMES, tt+delta_T)
            _, frame2 = cap.read()

            # Convert to grayscale
            gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]
            gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]

            # Normalize
            im1 = cv2.normalize(gray1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            im2 = cv2.normalize(gray2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

            # Compute the optical flow
            flow = optical_flow_tvl1(im1, im2)
            VX = -flow[..., 0] / delta_T
            VY = -flow[..., 1] / delta_T

            # Apply Gaussian smoothing if desired
            if smooth:
                VX = filters.gaussian(VX, sigma=smooth_len)
                VY = filters.gaussian(VY, sigma=smooth_len)

            # Subtract the mean
            VX -= VX.mean()
            VY -= VY.mean()

            print(f"Processed time step {tt}.")

        # Define the criteria for the ECC algorithm
        criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100, 0.001)

        # Initialize the warp matrix with identity transformation
        warp_matrix = np.eye(2, 3, dtype=np.float32)

        # Apply the ECC algorithm
        (cc, warp_matrix) = cv2.findTransformECC(im1, im2, warp_matrix, cv2.MOTION_EUCLIDEAN, criteria)

        # Use warpAffine to warp the second image
        im2_aligned = cv2.warpAffine(im2, warp_matrix, (im2.shape[1], im2.shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

        # Now let's calculate the optical flow between the original and warped images
        flow = cv2.calcOpticalFlowFarneback(im1, im2_aligned, None, pyr_scale=0.5, levels=5, winsize=11, iterations=5, poly_n=5, poly_sigma=1.2, flags=0)

        VX = flow[..., 0]
        VY = flow[..., 1]

        # Create PX and PY by element-wise multiplication of im1 and VX, VY respectively
        PX = im1 * VX
        PY = im1 * VY

        print(f"Shape of VX: {VX.shape}, Shape of VY: {VY.shape}")
        print(f"Shape of PX: {PX.shape}, Shape of PY: {PY.shape}")

        if smooth:
            VX_smooth = gaussian_filter(VX, smooth_len)
            VY_smooth = gaussian_filter(VY, smooth_len)
            print("VX and VY have been smoothed successfully.")
        else:
            VX_smooth = VX
            VY_smooth = VY
            print("No smoothing applied.")

        VX_mean_subtracted = VX_smooth - np.mean(VX_smooth)
        VY_mean_subtracted = VY_smooth - np.mean(VY_smooth)

        print("Mean subtracted from VX and VY successfully.")

        PX = im1 * VX_mean_subtracted
        PY = im1 * VY_mean_subtracted

        print("PX and PY have been calculated.")

        # Step 7: Resize and rescale UX and UY
        UX = cv2.resize(PX, (Y1.shape[1], X1.shape[0])) * vscale
        UY = cv2.resize(PY, (Y1.shape[1], X1.shape[0])) * vscale

        print("UX and UY have been resized and rescaled.")

        # Step 8: Calculate the mean, smoothed data, and product of
        PXmean = np.mean(PX, axis=0)
        PXsmth = gaussian_filter1d(PXmean, smooth_len)
        PXsmth = PXsmth / np.max(np.abs(PXsmth))

        PXprod = PXsmth[:-1] * PXsmth[1:]

        print("Calculated the mean, smoothed data, and product of smoothed data.")

        # Step 9: Find zero-crossings in PXprod
        PXzc = np.where(np.diff(np.sign(PXprod)))[0]

        print("Found zero-crossings in PXprod.")

        # Step 10: Calculate the first difference of PXsmth
        PXd1 = np.diff(PXsmth, n=1)

        print("Calculated the first difference of PXsmth.")

        # Step 11: Identify sinks
        PXsink = [ii for ii in PXzc if PXd1[ii] < 0]

        # Calculate the correlation distance between pixels
        if len(PXsink) > 1:
            corr_distance = np.mean(np.diff(PXsink))
        else:
            # If there's only one sink, correlation length is the width of the rectangle
            corr_distance = EdgeLength * pxlen

        corr_len[i] = corr_distance

        print(f"Identified {len(PXsink)} sinks.")

        # Step 12: Identify sources
        PXsrc = set(PXzc) - set(PXsink)
        PXsrc = [1] + list(PXsrc) + [max(xrange)]

        print(f"Identified {len(PXsrc)} sources.")

        # Step 13: Calculate peaks and wells
        peak_indices = []
        well_indices = []
        for ss in range(len(PXsrc) - 1):
            range_min, range_max = PXsrc[ss] - 1, PXsrc[ss + 1] - 1
            if range_min < range_max:
                peak_indices.append(np.argmax(PXsmth[range_min:range_max + 1]) + range_min)
                well_indices.append(np.argmin(PXsmth[range_min:range_max + 1]) + range_min)

        print(f"Identified {len(peak_indices)} peak indices and {len(well_indices)} well indices.")
        # Step 14: Calculate gaps
        gaps = [well_indices[ii] - peak_indices[ii] for ii in range(len(peak_indices)) if well_indices[ii] - peak_indices[ii] > 0]

        print(f"Identified {len(gaps)} gaps.")


In [ ]:
import cv2
import numpy as np
from skimage.registration import optical_flow_tvl1
from skimage import filters
from scipy.ndimage import gaussian_filter, gaussian_filter1d


In [ ]:
def process_frame(frame, xrange, yrange):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]
    return cv2.normalize(gray, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
def get_flow(im1, im2, delta_T, smooth=False, smooth_len=10):
    flow = optical_flow_tvl1(im1, im2)
    VX = -flow[..., 0] / delta_T
    VY = -flow[..., 1] / delta_T

    if smooth:
        VX = filters.gaussian(VX, sigma=smooth_len)
        VY = filters.gaussian(VY, sigma=smooth_len)

    VX -= VX.mean()
    VY -= VY.mean()

    return VX, VY
def get_corr_distance(PX, smooth_len, pxlen, EdgeLength):
    PXmean = np.mean(PX, axis=0)
    PXsmth = gaussian_filter1d(PXmean, smooth_len)
    PXsmth = PXsmth / np.max(np.abs(PXsmth))
    PXprod = PXsmth[:-1] * PXsmth[1:]
    PXzc = np.where(np.diff(np.sign(PXprod)))[0]
    PXd1 = np.diff(PXsmth, n=1)
    PXsink = [ii for ii in PXzc if PXd1[ii] < 0]

    if len(PXsink) > 1:
        corr_distance = np.mean(np.diff(PXsink))
    else:
        corr_distance = EdgeLength * pxlen

    return corr_distance
def get_correlation_len(tseries, cap, xrange, yrange, delta_T, EdgeLength, smooth, smooth_len, pxlen, vscale):
    all_corr_len = np.zeros(len(tseries))
    
    tp = 0
    for tt in tseries:
        tp += 1
        cap.set(cv2.CAP_PROP_POS_FRAMES, tt)
        _, frame1 = cap.read()
        cap.set(cv2.CAP_PROP_POS_FRAMES, tt+delta_T)
        _, frame2 = cap.read()

        im1 = process_frame(frame1, xrange, yrange)
        im2 = process_frame(frame2, xrange, yrange)

        # image registration
        moved, warp_matrix = cv2.findTransformECC(im1, im2, np.eye(2, 3, dtype=np.float32), cv2.MOTION_EUCLIDEAN)
        warp_matrix[0, 2] /= delta_T
        warp_matrix[1, 2] /= delta_T
        
        # Smoothing
        if smooth:
            warp_matrix[0, 2] = gaussian_filter(warp_matrix[0, 2], smooth_len)
            warp_matrix[1, 2] = gaussian_filter(warp_matrix[1, 2], smooth_len)
        
        # Subtract mean
        warp_matrix[0, 2] -= np.mean(warp_matrix[0, 2])
        warp_matrix[1, 2] -= np.mean(warp_matrix[1, 2])
        
        # Calculate P vectors
        PX = im1 * warp_matrix[0, 2]
        PY = im1 * warp_matrix[1, 2]

        # Rescale
        UX = vscale * cv2.resize(PX, (len(xrange), len(yrange)))
        UY = vscale * cv2.resize(PY, (len(xrange), len(yrange)))

        # Smooth the data and find zero-crossings
        PXmean = np.mean(PX, axis=0)
        PXsmth = gaussian_filter(PXmean, smooth_len)
        PXsmth = PXsmth / np.max(np.abs(PXsmth))
        PXprod = PXsmth[:-1] * PXsmth[1:]
        PXzc = np.where(PXprod<0)[0]
        
        # Find sink points
        PXd1 = np.diff(PXsmth)
        PXsink = [ii for ii in PXzc if PXd1[ii] < 0]

        # Find source points
        PXsrc = list(set(PXzc) - set(PXsink))
        PXsrc = [min(xrange)] + PXsrc + [max(xrange)]

        # Find peaks and wells
        peaks = []
        wells = []
        for ss in range(len(PXsrc)-1):
            peak = np.max(PXsmth[PXsrc[ss]:PXsrc[ss+1]])
            well = np.min(PXsmth[PXsrc[ss]:PXsrc[ss+1]])
            peaks.append(peak)
            wells.append(well)
        
        # Calculate correlation length
        gaps = [well - peak for well, peak in zip(wells, peaks) if well - peak > 0]
        corr_len = pxlen * np.mean(gaps)

        all_corr_len[tp] = corr_len
        
    return all_corr_len



In [ ]:
# Initial setup
saveimg = False
sample = '235_MT_correlation_15ul'
vidpath = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/'
vidname = vidpath + sample + '.avi'
EdgeLength = 40
smooth = 1
smooth_len = 10
pxlen = .43

delta_T = 1
vscale = 1
tseries = list(range(10, 101, 5))

cap = cv2.VideoCapture(vidname)
# read a frame from the video to get its dimensions
ret, frame = cap.read()
height, width, _ = frame.shape

# define xrange and yrange based on the dimensions of the video
xrange = range(width)
yrange = range(851, min(1051, height)) 

all_corr_len = get_correlation_len(tseries, cap, xrange, yrange, delta_T, EdgeLength, smooth, smooth_len, pxlen, vscale)



In [ ]:
plt.plot(tseries, all_corr_len, linewidth=5)
plt.xlabel('Time')
plt.ylabel('Correlation Length / um')

In [ ]:
# Initialize an array to store correlation lengths for all pairs of frames
all_corr_len = np.zeros((len(tseries), len(tseries)))

# Nested loop to calculate correlation lengths for all pairs of frames
for i, tt1 in enumerate(tseries):
    for j, tt2 in enumerate(tseries):
        if i == j:
            continue

        cap.set(cv2.CAP_PROP_POS_FRAMES, tt1)
        _, frame1 = cap.read()
        cap.set(cv2.CAP_PROP_POS_FRAMES, tt2)
        _, frame2 = cap.read()

        im1 = process_frame(frame1, xrange, yrange)
        im2 = process_frame(frame2, xrange, yrange)

        corr_len = get_correlation_len(tseries, cap, xrange, yrange, delta_T, EdgeLength, smooth, smooth_len, pxlen, vscale)
        all_corr_len[i, j] = corr_len


In [ ]:
import cv2
import numpy as np
from skimage.registration import optical_flow_tvl1
from skimage import filters
from scipy.ndimage import gaussian_filter, gaussian_filter1d

# Initial setup
saveimg = False
sample = '235_MT_correlation_15ul'
vidpath = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/'
vidname = vidpath + sample + '.avi'
EdgeLength = 40
smooth = True
smooth_len = 10
pxlen = .43
vscale = 1
tmin = 10
tmax = 100
delta_T = 1
step = 5
tseries = range(tmin, tmax+1, step)

# Open the video
cap = cv2.VideoCapture(vidname)

if not cap.isOpened():
    raise Exception(f"Could not open video file {vidname}")

# Define the range of the video we are interested in
ret, frame1 = cap.read()
yrange = range(851, 1051)
xrange = range(frame1.shape[1])

# Initialize correlation length array
corr_len = np.zeros(len(tseries))

# Loop over the time series
for i, tt in enumerate(tseries):
    # Read frames tt and tt+delta_T
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt)
    _, frame1 = cap.read()
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt+delta_T)
    _, frame2 = cap.read()

    # Convert to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]

    # Normalize
    im1 = cv2.normalize(gray1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    im2 = cv2.normalize(gray2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    # Compute the optical flow
    flow = optical_flow_tvl1(im1, im2)
    flow = np.transpose(flow, (1, 2, 0)) # Transpose the flow to have shape (200, 2048, 2)
    VX = -flow[..., 0] / delta_T
    VY = -flow[..., 1] / delta_T
    print("Optical flow (Python):")
    print(flow)


    # Apply Gaussian smoothing if desired
    if smooth:
        VX = filters.gaussian(VX, sigma=smooth_len)
        VY = filters.gaussian(VY, sigma=smooth_len)

    # Subtract the mean
    VX -= VX.mean()
    VY -= VY.mean()

    # ... Continue with the rest of your original code here
    # Continue from your last piece of code...

    # Calculate products
    PX = im1 * VX
    PY = im1 * VY
    print("Products (Python):")
    print(PX)
    print(PY)

    # Resize to the original image's size
    UX = cv2.resize(PX, (len(xrange), len(yrange)))
    UY = cv2.resize(PY, (len(xrange), len(yrange)))

    # Compute smoothed version of PX
    PXmean = np.mean(PX, axis=1)
    PXsmth = gaussian_filter1d(PXmean, smooth_len)
    PXsmth = PXsmth / np.abs(PXsmth).max()

    # Find zero-crossings
    PXprod = PXsmth[:-1] * PXsmth[1:]
    PXzc = np.where(PXprod < 0)[0]
    print("Zero-crossings (Python):")
    print(PXzc)

    # First derivative of smoothed PX
    PXd1 = np.gradient(PXsmth)
    print("Smoothed PX (Python):")
    print(PXsmth)

    # Find sinks
    PXsink = [ii for ii in PXzc if PXd1[ii] < 0]

    # Find sources
    PXsrc = list(set(PXzc) - set(PXsink))
    PXsource = [1] + PXsrc + [max(xrange)]
    
    # Find peaks and wells
    peaks = []
    wells = []
    for ss in range(1, len(PXsource)):
        slice_data = PXsmth[PXsource[ss - 1]:PXsource[ss]]
        if len(slice_data) > 0:
            valp, peak = max((val, idx) for (idx, val) in enumerate(slice_data))
            valw, well = min((val, idx) for (idx, val) in enumerate(slice_data))
            peaks.append(peak + PXsource[ss - 1])
            wells.append(well + PXsource[ss - 1])



    # Calculate gaps, correlation lengths
    gaps = np.array(wells) - np.array(peaks)
    gaps = gaps[gaps > 0]
    print("Gaps (Python):")
    print(gaps)
    corr_len[i] = pxlen * np.mean(gaps)


    np.savez(f'python/optical_flow_{tt}.npz', flow=flow)
    np.savez(f'python/products_{tt}.npz', PX=PX, PY=PY)
    np.savez(f'python/smoothed_PX_{tt}.npz', PXsmth=PXsmth)
    np.savez(f'python/zero_crossings_{tt}.npz', PXzc=PXzc)
    np.savez(f'python/gaps_{tt}.npz', gaps=gaps)

# After the loop
import matplotlib.pyplot as plt

plt.figure()
plt.plot(tseries, corr_len, linewidth=5)
plt.xlabel('Time')
plt.ylabel('Correlation Length / um')
plt.savefig('CorrLength.png')

# Prepare the time values and export to a CSV file
import pandas as pd

output_data = pd.DataFrame({'Time': tseries, 'Correlation Length': corr_len})
output_data.to_csv(sample + '_new.csv', index=False)


In [ ]:
corr_len

In [ ]:
from scipy.io import loadmat
matlab_path = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/matlab/'
python_path = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/python/'

# Rest of the comparison code remains the same...


for tt in tseries:
    # Load data from Python
    flow_python = np.load(f'{python_path}optical_flow_{tt}.npz')['flow']
    products_python = np.load(f'{python_path}products_{tt}.npz')
    smoothed_PX_python = np.load(f'{python_path}smoothed_PX_{tt}.npz')['PXsmth']
    zero_crossings_python = np.load(f'{python_path}zero_crossings_{tt}.npz')['PXzc']
    gaps_python = np.load(f'{python_path}gaps_{tt}.npz')['gaps']

    # Load data from MATLAB
    flow_matlab = loadmat(f'{matlab_path}optical_flow_{tt}.mat')['VV']
    products_matlab = loadmat(f'{matlab_path}products_{tt}.mat')
    smoothed_PX_matlab = loadmat(f'{matlab_path}smoothed_PX_{tt}.mat')['PXsmth']
    zero_crossings_matlab = loadmat(f'{matlab_path}zero_crossings_{tt}.mat')['PXzc']
    gaps_matlab = loadmat(f'{matlab_path}gaps_{tt}.mat')['gaps']



In [ ]:
    # Compare the data
    print(f"Time {tt}")
    print("Flow difference: ", np.sum(np.abs(flow_python - flow_matlab)))
    print("PX difference: ", np.sum(np.abs(products_python['PX'] - products_matlab['PX'])))
    print("PY difference: ", np.sum(np.abs(products_python['PY'] - products_matlab['PY'])))
    print("Smoothed PX difference: ", np.sum(np.abs(smoothed_PX_python - smoothed_PX_matlab)))
    print("Zero crossings difference: ", np.sum(np.abs(zero_crossings_python - zero_crossings_matlab)))
    print("Gaps difference: ", np.sum(np.abs(gaps_python - gaps_matlab)))
    print("\n")

In [ ]:
print("Shape of Smoothed PX (Python): ", smoothed_PX_python.shape)
print("Shape of Smoothed PX (MATLAB): ", smoothed_PX_matlab.shape)


In [ ]:
import cv2
import numpy as np
from skimage import filters
from scipy.ndimage import gaussian_filter, gaussian_filter1d
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pandas as pd

# Initial setup
saveimg = False
sample = '235_MT_correlation_15ul'
vidpath = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/'
vidname = vidpath + sample + '.avi'
EdgeLength = 40
smooth = True
smooth_len = 10
pxlen = .43
vscale = 1
tmin = 10
tmax = 100
delta_T = 1
step = 5
tseries = range(tmin, tmax+1, step)

# Open the video
cap = cv2.VideoCapture(vidname)

if not cap.isOpened():
    raise Exception(f"Could not open video file {vidname}")

# Define the range of the video we are interested in
ret, frame1 = cap.read()
yrange = range(851, 1051)
xrange = range(frame1.shape[1])

# Initialize correlation length array
corr_len = np.zeros(len(tseries))

# Loop over the time series
for i, tt in enumerate(tseries):
    # Read frames tt and tt+delta_T
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt)
    _, frame1 = cap.read()
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt+delta_T)
    _, frame2 = cap.read()

    # Convert to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)[yrange, :][:, xrange]

    # Normalize
    im1 = cv2.normalize(gray1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    im2 = cv2.normalize(gray2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Convert the grayscale images to SimpleITK format
    fixed_image = sitk.GetImageFromArray(im1)
    moving_image = sitk.GetImageFromArray(im2)

    # Perform image registration using Demons algorithm
    demons_filter = sitk.DemonsRegistrationFilter()
    demons_filter.SetNumberOfIterations(50)

    # You can set standard deviation for Gaussian smoothing of displacement field
    demons_filter.SetStandardDeviations(1.0)

    displacement_field = demons_filter.Execute(fixed_image, moving_image)

    # Convert the displacement field to a numpy array
    displacement_field = sitk.GetArrayFromImage(displacement_field)

    # Extract the velocity components
    VX = -displacement_field[..., 0] / delta_T
    VY = -displacement_field[..., 1] / delta_T

    print("Displacement field (Python):")
    print(displacement_field)

    # Apply Gaussian smoothing if desired
    if smooth:
        VX = filters.gaussian(VX, sigma=smooth_len)
        VY = filters.gaussian(VY, sigma=smooth_len)

    # Subtract the mean
    VX -= VX.mean()
    VY -= VY.mean()

    # Calculate products
    PX = im1 * VX
    PY = im1 * VY
    print("Products (Python):")
    print(PX)
    print(PY)

    # Compute smoothed version of PX
    PXmean = np.mean(PX, axis=1)
    PXsmth = gaussian_filter1d(PXmean, smooth_len)
    PXsmth = PXsmth / np.abs(PXsmth).max()

    # Find zero-crossings
    PXprod = PXsmth[:-1] * PXsmth[1:]
    PXzc = np.where(PXprod < 0)[0]
    print("Zero-crossings (Python):")
    print(PXzc)

    # First derivative of smoothed PX
    PXd1 = np.gradient(PXsmth)
    print("Smoothed PX (Python):")
    print(PXsmth)

    # Find sinks
    PXsink = [ii for ii in PXzc if PXd1[ii] < 0]

    # Find sources
    PXsrc = list(set(PXzc) - set(PXsink))
    PXsource = [1] + PXsrc + [max(xrange)]
    
    # Find peaks and wells
    peaks = []
    wells = []
    for ss in range(1, len(PXsource)):
        slice_data = PXsmth[PXsource[ss - 1]:PXsource[ss]]
        if len(slice_data) > 0:
            valp, peak = max((val, idx) for (idx, val) in enumerate(slice_data))
            valw, well = min((val, idx) for (idx, val) in enumerate(slice_data))
            peaks.append(peak + PXsource[ss - 1])
            wells.append(well + PXsource[ss - 1])

    # Calculate gaps, correlation lengths
    gaps = np.array(wells) - np.array(peaks)
    gaps = gaps[gaps > 0]
    print("Gaps (Python):")
    print(gaps)
    corr_len[i] = pxlen * np.mean(gaps)

    np.savez(f'python/optical_flow_{tt}.npz', flow=flow)
    np.savez(f'python/products_{tt}.npz', PX=PX, PY=PY)
    np.savez(f'python/smoothed_PX_{tt}.npz', PXsmth=PXsmth)
    np.savez(f'python/zero_crossings_{tt}.npz', PXzc=PXzc)
    np.savez(f'python/gaps_{tt}.npz', gaps=gaps)

# After the loop
plt.figure()
plt.plot(tseries, corr_len, linewidth=5)
plt.xlabel('Time')
plt.ylabel('Correlation Length / um')
plt.savefig('CorrLength.png')

# Prepare the time values and export to a CSV file
output_data = pd.DataFrame({'Time': tseries, 'Correlation Length': corr_len})
output_data.to_csv(sample + '_new.csv', index=False)


# Real Correlation Distance

In [ ]:
from scipy.ndimage import gaussian_filter
import cv2
from skimage.registration import optical_flow_tvl1
import cupy as cp

def calculate_PIV(frame1, frame2, smooth=True, smooth_len=10, delta_T=1):
    # Convert to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Normalize the images
    im1 = cv2.normalize(gray1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    im2 = cv2.normalize(gray2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Compute the optical flow
    flow = optical_flow_tvl1(im1, im2)
    VX = -flow[..., 0] / delta_T
    VY = -flow[..., 1] / delta_T

    # Apply Gaussian smoothing if desired
    if smooth:
        VX = gaussian_filter(VX, sigma=smooth_len)
        VY = gaussian_filter(VY, sigma=smooth_len)

    # Subtract the mean
    VX -= VX.mean()
    VY -= VY.mean()

    return VX, VY


In [ ]:
if not cv2.cuda.getCudaEnabledDeviceCount():
    raise Exception("No CUDA-enabled GPU found!")

In [ ]:
import cv2
import cupy as cp
from skimage.registration import optical_flow_tvl1
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


sample = '235_MT_correlation_15ul'
vidpath = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/'
vidname = vidpath + sample + '.avi'
delta_T = 1
tmin = 10
tmax = 100
step = 5
tseries = range(tmin, tmax+1, step)

# Open the video
cap = cv2.VideoCapture(vidname)

if not cap.isOpened():
    raise Exception(f"Could not open video file {vidname}")

# Initialize the PIV data list
piv_data = []

# Create progress bar
pbar = tqdm(total=len(tseries[:-1]))

# Loop over the time series
for i, tt in enumerate(tseries[:-1]):
    # Read frames tt and tt+delta_T
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt)
    _, frame1 = cap.read()
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt+delta_T)
    _, frame2 = cap.read()

    # Convert frames to grayscale if they're not already
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Transfer data to the GPU
    gray1_gpu = cp.asarray(gray1)
    gray2_gpu = cp.asarray(gray2)

    # Compute the optical flow with the skimage function
    # Note: skimage works on the CPU, so we need to transfer the data back from the GPU
    flow = optical_flow_tvl1(cp.asnumpy(gray1_gpu), cp.asnumpy(gray2_gpu))

    # Transfer results back to GPU memory
    flow_gpu = cp.asarray(flow)
    VX = flow_gpu[..., 0]
    VY = flow_gpu[..., 1]

    # Transfer results back to CPU memory and append to the data list
    piv_data.append((cp.asnumpy(VX), cp.asnumpy(VY)))

    # Update progress bar
    pbar.update(1)

# Close progress bar
pbar.close()

print("PIV data has been computed.")


In [ ]:
# Select the first PIV data point
VX, VY = piv_data[0]

# Make a grid of points
X, Y = np.meshgrid(np.arange(0, VX.shape[1], 1), np.arange(0, VX.shape[0], 1))

# Downsample the data to make the plot manageable
# (Adjust as needed for your data)
X = X[::10, ::10]
Y = Y[::10, ::10]
VX = VX[::10, ::10]
VY = VY[::10, ::10]

# Create a quiver plot
plt.figure(figsize=(10,10))
plt.quiver(X, Y, VX, VY, angles='xy', scale_units='xy', scale=1, width=0.005)
plt.gca().invert_yaxis()  # invert the y-axis to match the image orientation
plt.show()

In [ ]:
import matplotlib.pyplot as plt

def visualize_piv(frame, VX, VY, step=10):
    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Create a grid of points
    y, x = np.mgrid[0:VX.shape[0], 0:VX.shape[1]]

    # Plot the original frame
    plt.imshow(frame_rgb)

    # Plot the vector field
    plt.quiver(x[::step, ::step], y[::step, ::step], VX[::step, ::step], VY[::step, ::step], color='r')

    plt.show()

# Now, let's visualize a random frame's flow field
frame_idx = 5 # Choose an index

# Read the frame at time tseries[frame_idx]
cap.set(cv2.CAP_PROP_POS_FRAMES, tseries[frame_idx])
_, frame = cap.read()

# Get the corresponding PIV data
VX, VY = piv_data[frame_idx]

# Visualize the PIV data
visualize_piv(frame, VX, VY)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Select the first PIV data point
VX, VY = piv_data[0]

# Load the first frame of the video
cap = cv2.VideoCapture(vidname)
ret, frame = cap.read()
cap.release()
if not ret:
    raise Exception("Could not load the video frame")

# Convert to grayscale for plotting
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Make a grid of points
X, Y = np.meshgrid(np.arange(0, VX.shape[1], 1), np.arange(0, VX.shape[0], 1))

# Downsample the data to make the plot manageable
# (Adjust as needed for your data)
X = X[::10, ::10]
Y = Y[::10, ::10]
VX = VX[::10, ::10]
VY = VY[::10, ::10]

# Create a figure
plt.figure(figsize=(10,10))

# Display the grayscale image
plt.imshow(gray, cmap='gray')

# Overlay the quiver plot
plt.quiver(X, Y, VX, VY, angles='xy', scale_units='xy', scale=1, width=0.005, color='r')
plt.gca().invert_yaxis()  # invert the y-axis to match the image orientation
plt.show()


In [ ]:
# Create a figure
plt.figure(figsize=(10,10))

# Display the grayscale image
plt.imshow(gray, cmap='gray')

# Overlay the quiver plot
plt.quiver(X, Y, VX, VY, angles='xy', scale_units='xy', scale=0.1, width=0.005, color='r')
plt.gca().invert_yaxis()  # invert the y-axis to match the image orientation
plt.show()


In [ ]:
import numpy as np

# Define the number of vectors per row and column
num_vectors_x = VX.shape[1]
num_vectors_y = VX.shape[0]

# Create the 1D position arrays
x_positions = np.linspace(0, gray.shape[1], num_vectors_x)
y_positions = np.linspace(0, gray.shape[0], num_vectors_y)

# Create 2D position arrays
X, Y = np.meshgrid(x_positions, y_positions)

# Now, use these 2D position arrays in your quiver plot:
plt.figure(figsize=(10,10))
plt.imshow(gray, cmap='gray')
plt.quiver(X, Y, VX, VY, angles='xy', scale_units='xy', scale=1, color='r')
plt.gca().invert_yaxis()
plt.show()


In [ ]:
import numpy as np
import cv2
import cupy as cp
import matplotlib.pyplot as plt
from tqdm import tqdm

sample = '235_MT_correlation_15ul'
vidpath = '/home/shichen/Dropbox/Academics/PhD_phase/Thompson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/'
vidname = vidpath + sample + '.avi'

cap = cv2.VideoCapture(vidname)
ret, frame1 = cap.read()
gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Initialize the PIV data list
piv_data = []

# Loop over the time series
tseries = np.arange(0, 1000, 2)  # example time series
delta_T = 1
smooth = True
smooth_len = 1

# Create a progress bar
pbar = tqdm(total=len(tseries[:-1]))

for i, tt in enumerate(tseries[:-1]):
    # Read frames tt and tt+delta_T
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt)
    _, frame1 = cap.read()
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt+delta_T)
    _, frame2 = cap.read()

    # Compute the optical flow and store it in the piv_data list
    VX, VY = calculate_PIV_gpu(frame1, frame2, smooth=smooth, smooth_len=smooth_len, delta_T=delta_T)
    piv_data.append((VX, VY))
    pbar.update(1)

pbar.close()

# convert to numpy arrays for plotting
VX, VY = piv_data[0]

# Define the number of vectors per row and column
num_vectors_y = VX.shape[0]
num_vectors_x = VX.shape[1]

# Create the 1D position arrays
y_positions = np.linspace(0, gray.shape[0], num_vectors_y)
x_positions = np.linspace(0, gray.shape[1], num_vectors_x)

# Create 2D position arrays
Y, X = np.meshgrid(y_positions, x_positions)

# Now, use these 2D position arrays in your quiver plot:
plt.figure(figsize=(10,10))
plt.imshow(gray, cmap='gray')
plt.quiver(X, Y, VX.get(), VY.get(), angles='xy', scale_units='xy', scale=1, color='r')
plt.gca().invert_yaxis()
plt.show()


In [ ]:
cap = cv2.VideoCapture(vidname)

ret = False
while not ret:
    ret, frame1 = cap.read()
    if ret:
        gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    else:
        print("Warning: frame not read. Trying again...")


In [ ]:
!pip install openpiv


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from skimage.registration import optical_flow_tvl1

# Parameters
gridSize = 30
smooth = 1
sigma = 30
velScale = 1.6

sample = '235_MT_correlation_15ul'
vidpath = '/home/shichen/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/'
vidname = vidpath + sample + '.avi'

# Open video
cap = cv2.VideoCapture(vidname)
Ttot = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
delta_T = 1
step = 1
tseries = range(0, Ttot-1, step)

_, im1 = cap.read()  # Read the first frame to determine the image size
xrange = range(im1.shape[1])
yrange = range(im1.shape[0])

X1, Y1 = np.meshgrid(np.arange(gridSize/2, len(xrange)-gridSize/2, gridSize), 
                     np.arange(gridSize/2, len(yrange)-gridSize/2, gridSize))

for tt in tseries:
    cap.set(cv2.CAP_PROP_POS_FRAMES, tt)
    _, im1 = cap.read()
    _, im2 = cap.read()

    im1 = im1[np.ix_(yrange, xrange)]
    im2 = im2[np.ix_(yrange, xrange)]

    # Compute PIV using optical_flow_tvl1
    flow = optical_flow_tvl1(cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY), cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY))
    VX = -flow[..., 0]
    VY = -flow[..., 1]

    if smooth:
        VX = gaussian_filter(VX, sigma=sigma) / delta_T
        VY = gaussian_filter(VY, sigma=sigma) / delta_T

    UX = cv2.resize(VX, (X1.shape[1], X1.shape[0])) * velScale
    UY = cv2.resize(VY, (X1.shape[1], X1.shape[0])) * velScale

    # Visualize
    plt.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY), cmap='gray')
    plt.quiver(X1, Y1, UX, UY, color='c', scale=25)
    plt.gca().set_facecolor('black')
    plt.axis('equal')
    plt.show()

cap.release()


In [ ]:
cap

In [ ]:
Ttot

In [ ]:
# Calculate magnitude of velocity
magnitude = np.sqrt(VX**2 + VY**2)

# Display the magnitude
plt.imshow(magnitude, cmap='viridis')
plt.colorbar(label='Velocity Magnitude')
plt.title("Magnitude of Velocity")
plt.show()


In [ ]:
VX

In [ ]:
VY

In [ ]:
plt.hist(VX.flatten(), bins=50)
plt.title('Histogram of VY values')
plt.xlabel('VY value')
plt.ylabel('Frequency')
plt.show()


In [ ]:
average_vy_values = [np.mean(data[1]) for data in piv_data]

plt.plot(tseries[:-1], average_vy_values)
plt.xlabel('Time Step')
plt.ylabel('Average VY Value')
plt.title('Average VY value over Time')
plt.grid(True)
plt.show()
